In [1]:
!pip install torch transformers datasets nltk tqdm rouge

In [2]:
import torch
from transformers import BartForConditionalGeneration, BartTokenizer, Trainer, TrainingArguments, DataCollatorForSeq2Seq,GenerationConfig
from datasets import load_dataset
from torch.utils.data import DataLoader
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re
import multiprocessing
from tqdm import tqdm
from rouge import Rouge

import multiprocessing
multiprocessing.set_start_method("forkserver", force=True)

In [3]:
# Define contraction mapping (you can expand this as needed)
contraction_mapping = {
    "ain't": "is not", "aren't": "are not", "can't": "cannot", "'cause": "because", "could've": "could have",
    "couldn't": "could not", "didn't": "did not", "doesn't": "does not", "don't": "do not", "hadn't": "had not",
    "hasn't": "has not", "haven't": "have not", "he'd": "he would", "he'll": "he will", "he's": "he is",
    "how'd": "how did", "how'll": "how will", "how's": "how is", "I'd": "I would", "I'll": "I will",
    "I'm": "I am", "I've": "I have", "isn't": "is not", "it's": "it is", "let's": "let us", "ma'am": "madam",
    "mightn't": "might not", "might've": "might have", "mustn't": "must not", "must've": "must have",
    "needn't": "need not", "o'clock": "of the clock", "should've": "should have", "shouldn't": "should not",
    "that'd": "that would", "that's": "that is", "there's": "there is", "they'd": "they would", "they'll": "they will",
    "they're": "they are", "they've": "they have", "wasn't": "was not", "we'd": "we would", "we'll": "we will",
    "we're": "we are", "we've": "we have", "weren't": "were not", "what's": "what is", "where's": "where is",
    "who's": "who is", "won't": "will not", "would've": "would have", "wouldn't": "would not", "y'all": "you all",
    "you're": "you are", "you've": "you have"
}
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [4]:
# Function to preprocess and clean text
def preprocess_and_clean(text):
    text = re.sub(r'[^A-Za-z0-9]', ' ', text)  # Remove non-alphanumeric characters
    text = text.lower()  # Convert text to lowercase
    text = ' '.join([contraction_mapping.get(t, t) for t in text.split()])  # Expand contractions
    tokens = word_tokenize(text)
    stop_words = set(stopwords.words('english'))
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(token) for token in tokens if token not in stop_words]
    return ' '.join(tokens)

In [5]:
# Function to preprocess a dataset in parallel
def preprocess_dataset(dataset):
    num_cpus = multiprocessing.cpu_count()
    pool = multiprocessing.Pool(processes=num_cpus)
    processed_texts = []
    with tqdm(total=len(dataset), desc="Preprocessing") as pbar:
        for cleaned_text in pool.imap_unordered(preprocess_and_clean, dataset):
            processed_texts.append(cleaned_text)
            pbar.update()
    pool.close()
    pool.join()
    return processed_texts

In [6]:
# Load the dataset (for example, CNN/Daily Mail)
dataset = load_dataset('cnn_dailymail', '3.0.0')

# Limit the dataset to a smaller subset (e.g., first 1000 examples)
limited_train_dataset = dataset['train']
limited_eval_dataset = dataset['validation']
limited_test_dataset = dataset['test'].select(range(500))

README.md:   0%|          | 0.00/15.6k [00:00<?, ?B/s]

train-00000-of-00003.parquet:   0%|          | 0.00/257M [00:00<?, ?B/s]

train-00001-of-00003.parquet:   0%|          | 0.00/257M [00:00<?, ?B/s]

train-00002-of-00003.parquet:   0%|          | 0.00/259M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/34.7M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/30.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/287113 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/13368 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11490 [00:00<?, ? examples/s]

In [7]:
# Tokenize the dataset for the BART model
def preprocess_data(examples):
    model_inputs = bart_tokenizer(
        examples['article'], max_length=1024, truncation=True, padding="max_length"
    )
    
    labels = bart_tokenizer(
            examples['highlights'], max_length=128, truncation=True, padding="max_length"
        )
    model_inputs['labels'] = labels['input_ids']
    return model_inputs

In [8]:
# Initialize the BART tokenizer
bart_tokenizer = BartTokenizer.from_pretrained('facebook/bart-large-cnn')

# Apply preprocessing
tokenized_datasets = {
    'train': limited_train_dataset.map(preprocess_data, batched=True),
    'validation': limited_eval_dataset.map(preprocess_data, batched=True),
    'test': limited_test_dataset.map(preprocess_data, batched=True)
}

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

Map:   0%|          | 0/287113 [00:00<?, ? examples/s]

Map:   0%|          | 0/13368 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

In [9]:
# Load the BART model
bart_model = BartForConditionalGeneration.from_pretrained('facebook/bart-large-cnn')
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(torch.cuda.is_available())  # Check if GPU is available
bart_model.to(device)

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

True


BartForConditionalGeneration(
  (model): BartModel(
    (shared): BartScaledWordEmbedding(50264, 1024, padding_idx=1)
    (encoder): BartEncoder(
      (embed_tokens): BartScaledWordEmbedding(50264, 1024, padding_idx=1)
      (embed_positions): BartLearnedPositionalEmbedding(1026, 1024)
      (layers): ModuleList(
        (0-11): 12 x BartEncoderLayer(
          (self_attn): BartSdpaAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=4096, out_features=1024, bias=True)
    

In [10]:
# Data collator for the model
data_collator = DataCollatorForSeq2Seq(tokenizer=bart_tokenizer, model=bart_model, pad_to_multiple_of=8)

# Training arguments
training_args = TrainingArguments(
    output_dir='./bart_finetuned',
    eval_strategy="steps",
    learning_rate=5e-5,  # Keep the learning rate the same
    per_device_train_batch_size=1,  # Keeping this as 1 for memory efficiency
    gradient_accumulation_steps=2,  # Accumulate gradients to simulate larger batch size
    per_device_eval_batch_size=1,
    save_steps=500,  # Save less frequently to reduce I/O time
    max_steps=500,  # Reduced the number of steps to speed up training
    num_train_epochs=1,  # Set epochs to 1 for faster training
    save_total_limit=2,
    logging_dir='./logs',
    logging_steps=100,  # Keep the logging frequency to monitor training
    save_strategy="epoch",  # Save at the end of each epoch
    fp16=True,  # Mixed precision training for faster training
    report_to="none",
    no_cuda=False if torch.cuda.is_available() else True,
    tpu_num_cores=None,
    dataloader_num_workers=4,  # Number of workers for faster data loading
    lr_scheduler_type='linear',  # Apply linear learning rate scheduler
    warmup_steps=200,  # Reduced warmup steps for quicker training
)


# Trainer for fine-tuning
trainer = Trainer(
    model=bart_model,
    args=training_args,
    train_dataset=tokenized_datasets['train'].shuffle(seed=42).select(range(10000)),
    eval_dataset=tokenized_datasets['validation'].shuffle(seed=42).select(range(1000)),
    processing_class=bart_tokenizer,
    data_collator=data_collator,
)
# Define generation configuration
generation_config = GenerationConfig(
    max_length=142,
    min_length=50,
    early_stopping=True,
    num_beams=4,
    length_penalty=2.0,
    no_repeat_ngram_size=3,
    forced_bos_token_id=0,
)

# Train the model
trainer.train()

max_steps is given, it will override any value given in num_train_epochs


Step,Training Loss,Validation Loss
100,3.323200,0.935004
200,0.779900,0.984972
300,0.830300,0.934897
400,0.767200,0.890566
500,0.743400,0.869061


/opt/conda/lib/python3.10/site-packages/transformers/modeling_utils.py:2817: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 142, 'min_length': 56, 'early_stopping': True, 'num_beams': 4, 'length_penalty': 2.0, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


TrainOutput(global_step=500, training_loss=1.2887909698486328, metrics={'train_runtime': 1155.9947, 'train_samples_per_second': 0.865, 'train_steps_per_second': 0.433, 'total_flos': 2167104602112000.0, 'train_loss': 1.2887909698486328, 'epoch': 0.1})

In [11]:
# Evaluate the model
print("Evaluating the model...")

# Create a DataLoader for the test set
batch_size = 8  # Adjust based on your GPU memory
test_loader = DataLoader(tokenized_datasets['test']['article'], batch_size=batch_size)

predicted_summaries = []

for batch in test_loader:
    # Tokenize and move to device
    inputs = bart_tokenizer(
        batch, return_tensors="pt", max_length=512, truncation=True, padding=True
    )
    inputs = {key: value.to(device) for key, value in inputs.items()}
    
    # Generate summaries
    summary_ids = bart_model.generate(
        inputs['input_ids'], attention_mask=inputs['attention_mask'],
        generation_config=generation_config
    )
    
    # Decode and append summaries
    predicted_summaries.extend(bart_tokenizer.batch_decode(summary_ids, skip_special_tokens=True))

Evaluating the model...


In [12]:
# Calculate ROUGE scores
rouge = Rouge()
scores = rouge.get_scores(predicted_summaries, tokenized_datasets['test']['highlights'], avg=True)
print("ROUGE Scores:", scores)

ROUGE Scores: {'rouge-1': {'r': 0.40531385406904685, 'p': 0.25765859422834314, 'f': 0.3090104882724774}, 'rouge-2': {'r': 0.16261878606423094, 'p': 0.09542072287746495, 'f': 0.1176961824102381}, 'rouge-l': {'r': 0.3768242519603926, 'p': 0.23899180086990043, 'f': 0.28688761201064555}}


In [13]:
# Save the fine-tuned model
bart_model.save_pretrained('./bart_finetuned')
bart_tokenizer.save_pretrained('./bart_finetuned')

print("Fine-tuned model saved in './bart_finetuned'")

Fine-tuned model saved in './bart_finetuned'


In [14]:
# User-defined function for summarization
def summarize_text(text, num_words):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    bart_model.to(device) 
    min_length = min(30, num_words // 2)  
    max_length = num_words
    inputs = bart_tokenizer(text, return_tensors='pt', max_length=1024, truncation=True).to(device)
      # Generate summary
    summary_ids = bart_model.generate(
        inputs['input_ids'], 
        max_length=num_words, 
        num_beams=4, 
        early_stopping=True
    )
    
    # Decode and return the summary
    return bart_tokenizer.decode(summary_ids[0], skip_special_tokens=True)

In [15]:
# User input for testing
user_input = input("Enter the text to summarize: ")
num_words = 50
summary = summarize_text(user_input, num_words)
print("Input Text:", user_input)
print("Generated Summary:", summary)

Enter the text to summarize:  Artificial intelligence (AI) has become an integral part of modern technology, influencing industries ranging from healthcare to transportation. By leveraging machine learning algorithms, AI systems can analyze large datasets, recognize patterns, and make predictions with remarkable accuracy. In healthcare, for example, AI aids in diagnosing diseases, personalizing treatment plans, and even predicting patient outcomes. Meanwhile, in transportation, self-driving cars and intelligent traffic management systems are reshaping the way people and goods move. Despite its advantages, AI also raises concerns about privacy, job displacement, and ethical considerations, prompting ongoing debates about how best to balance innovation with responsibility.


/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1399: UserWarning: Unfeasible length constraints: `min_length` (56) is larger than the maximum possible length (50). Generation will stop at the defined maximum length. You should decrease the minimum length and/or increase the maximum length.
  warnings.warn(


Input Text: Artificial intelligence (AI) has become an integral part of modern technology, influencing industries ranging from healthcare to transportation. By leveraging machine learning algorithms, AI systems can analyze large datasets, recognize patterns, and make predictions with remarkable accuracy. In healthcare, for example, AI aids in diagnosing diseases, personalizing treatment plans, and even predicting patient outcomes. Meanwhile, in transportation, self-driving cars and intelligent traffic management systems are reshaping the way people and goods move. Despite its advantages, AI also raises concerns about privacy, job displacement, and ethical considerations, prompting ongoing debates about how best to balance innovation with responsibility.
Generated Summary: Artificial intelligence (AI) has become an integral part of modern technology, influencing industries ranging from healthcare to transportation .
By leveraging machine learning algorithms, AI systems can analyze large